In [ ]:
#C1
# Importar librerias

import numpy as np
import matplotlib.pyplot as plt
import time

import keras


# Librerias para la red convolucional
from keras.models import *
from keras.layers import *
from keras.optimizers import *

# Librerias para cargar imagenes 

from PIL import Image
import glob

In [ ]:
# C2
# Cargar la base de datos

# AigleRN - 38 imagenes

from google.colab import drive
drive.mount('/content/drive')

#Cargar la base de datos, directorio dependiente de cada persona
dir_img = '/content/drive/My Drive/Bases_de_datos/AigleRN/images'
dir_lab = '/content/drive/My Drive/Bases_de_datos/AigleRN/labels'

image_list = []
label_list = []

background_color = np.array([255, 0, 255])  ## Solo el magenta

for filename in glob.glob('/content/drive/My Drive/Bases_de_datos/AigleRN/images/*.png'): 
    im=Image.open(filename)
    image_list.append(np.asarray(im))
    #image_list.append(np.asarray(np.expand_dims(im, axis=0)))
    

    filename1 = filename.replace('images','labels')
    filename2 = filename1.replace('umm','umm_road')
    la=Image.open(filename2)

    gt_image = np.asarray(la)
    gt_bg = np.all(gt_image == background_color, axis=2)  #Binarizar la segmentación del target
    label_list.append(np.expand_dims(gt_bg,axis=2))


    #label_list.append(np.asarray(np.expand_dims(la, axis=0)))

    


print ('Total de imagenes: ', len(image_list))
print ('Total de labels: ', len(label_list))

In [ ]:
# C3
# Visualizar y acomodar los datos

# Visualizar los datos
fig = plt.figure()
for i in range (0,6):
  a = fig.add_subplot(2,3,i+1)
  plt.imshow(image_list[i])
plt.show()

fig1 = plt.figure()
for j in range (0,6):
  b = fig1.add_subplot(2,3,j+1)
  plt.imshow(label_list[j][:,:,0])
plt.show()


# Separar los datos en conjunto de entrenamiento y prueba

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(image_list, label_list, test_size=0.25, random_state=42)  # Se requiere el 75% para entrenamiento


print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

In [ ]:
# C4
# Modelo reducido

# Modelo: U-Net-B
input_size = (320,480,3)

def unetB(input_size = (320,480,3)):

    inputs = Input(input_size)
    s = Lambda(lambda x: x / 255) (inputs)
    
    conv1 = Conv2D(64, (3, 3), padding='same', activation='relu')(s)
    conv2 = Conv2D(64, (3, 3), padding='same', activation='relu')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Conv2D(128, (3, 3), padding='same', activation='relu')(pool1)
    conv4 = Conv2D(128, (3, 3), padding='same', activation='relu')(conv3)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    conv5 = Conv2D(256, (3, 3), padding='same', activation='relu')(pool2)
    conv6 = Conv2D(256, (3, 3), padding='same', activation='relu')(conv5)
    
    
    convT3 = Conv2D(128, (2, 2), padding='same', activation='relu')(UpSampling2D(size=(2, 2))(conv6))
    merge3 = concatenate([convT3, conv4], axis=3)
    conv15 = Conv2D(128, (3, 3), padding='same', activation='relu')(merge3)
    conv16 = Conv2D(128, (3, 3), padding='same', activation='relu')(conv15)
    
    convT4 = Conv2D(64, (2, 2), padding='same', activation='relu')(UpSampling2D(size=(2, 2))(conv16))
    merge4 = concatenate([convT4, conv2], axis=3)
    conv17 = Conv2D(64, (3, 3), padding='same', activation='relu')(merge4)
    conv18 = Conv2D(64, (3, 3), padding='same', activation='relu')(conv17)
    
    
    y = Conv2D(1, (1, 1), activation='sigmoid')(conv18)

    return Model(inputs=inputs, outputs=y)

model = unetB(input_size)

model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
#model.summary()

In [ ]:
# C5
# Entrenar el modelo

model.fit(x=np.array(X_train)
          ,y=np.array(y_train)
          ,epochs=___      # Definir el numero de epocas para el entrenamiento
          ,batch_size=2
          ,validation_data=(np.array(X_train),np.array(y_train)))

In [ ]:
# C6
# Probar el modelo con una sola imagen

plt.imshow(X_test[0])
plt.show()

t_l = y_test [0]
t1 = t_l[:,:,0]

plt.imshow(t1)
plt.show()

inicio = time.time()
pred = model.predict(np.expand_dims(X_test[0],axis=0))
fin = time.time()
print ('Tiempo de ejecucion: ', fin-inicio)


plt.imshow(pred[0,:,:,0])
plt.show()